# TA Application

Adding technical analysis to data + short summary of each type of indicator

## Indicators used in Coin Bureau video: 
[video](https://www.youtube.com/watch?v=lW3eWIj3Q04)

- BTC stock to flow
- BTC dominance
- Price patterns and triangle patterns (support and resistance)
- Moving average (prefer EMA and 200 day EMA)
- MACD (cross, etc)
- RSI (current and trend)
- Bollinger bands (total width)

## From [the trading channel](https://www.youtube.com/watch?v=eynxyoKgpng)
- For an uptrend, price does not go below most recent pullback. After an inpulse up, the pullback doesn't go below the previous pullback. 
- Support and resistance. Probably with very short moving average, check if \/ exists and /\. 
- Trade from support up to previous high in uptrend
- ATR indicator (Average true range)
    - Make sure stop loss is 1 ATR below dip and move 2x (or more) ATR 
- Moving average (20, 50, 200)
- RSI (best combined with Major levels of structure, trends on higher time frames, 

## Notes
Figure out how to do:
- Find trends (ie take derivative of moving average) Ans: Compare past value and current value and find difference
- 

In [6]:
## Imports 
import pandas as pd
from ta.utils import dropna
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import SMAIndicator, macd_diff
from ta.momentum import RSIIndicator

df = pd.read_csv('data/BTCUSDT-day.csv', sep=',')
df = df.drop('ignore', axis=1)
df = dropna(df)

In [7]:
# Simple moving indicator
def get_sma(p=[20, 50, 200], get_diff=True, get_rate=True):
    """
    adds sma indicators to df
    p is period, recommended is [20, 50, 200]
    """
    for i in p:
        sma = SMAIndicator(close=df["close"], window=i)
        name = "sma_" + str(i)
        df[name]=sma.sma_indicator()
    if get_diff:
        get_sma_diff(p)
    if get_rate:
        get_sma_rate(p)

In [8]:
# Simple moving indicator difference from close
def get_sma_diff(p=[20, 50, 200]):
    """
    adds sma-close
    """
    for i in p:
        c = df["close"]
        sma = SMAIndicator(close=c, window=i)
        s = sma.sma_indicator()
        result = list(map(lambda x, y: x - y, c, s))
        name = "sma_diff_" + str(i)
        df[name] = result

In [9]:
# SMA rate
def get_sma_rate(p=[20, 50, 200]):
    """
    adds sma rate by taking difference: close-previous_close (for SMAs)
    """
    for i in p:
        sma = SMAIndicator(close=df["close"], window=i)
        now = sma.sma_indicator()
        past = now.shift(1)
        name = "rate_sma_" + str(i)
        df[name] = list(map(none_subtraction, now, past))
        
def none_subtraction(x, y):
    """
    returns x-y and None if either are None
    """
    if x is None or y is None:
        return None
    else:
        return x-y

In [10]:
# MACD indicator
def get_macd(slow=26, fast=12, signal=9):
    """
    adds macd indicator
    """
    df["macd"] = macd_diff(close=df["close"], window_slow=slow, window_fast=fast, window_sign=signal)

In [11]:
# RSI indicator
def get_rsi(p=14):
    """
    adds rsi indicator
    p is period, default 14
    """
    rsi = RSIIndicator(close=df["close"], window=p)
    df["rsi"] = rsi.rsi()


In [12]:
# Bollinger bands
def get_bb(p=20, sd=2):
    """
    adds all bollinger band indicators to df
    p is period, default is 20
    sd is standard deviation, default is 2
    """
    bb = BollingerBands(close=df["close"], window=p, window_dev=sd)
    # Add Bollinger Bands features
    df['bb_bbm'] = bb.bollinger_mavg()   #Bollinger Channel Middle Band
    df['bb_bbh'] = bb.bollinger_hband()
    df['bb_bbl'] = bb.bollinger_lband()

    # Add Bollinger Band high indicator
    df['bb_bbhi'] = bb.bollinger_hband_indicator() # return 1 if close > hband else 0

    # Add Bollinger Band low indicator
    df['bb_bbli'] = bb.bollinger_lband_indicator()

    # Add Width Size Bollinger Bands
    df['bb_bbw'] = bb.bollinger_wband() # Bollinger Channel Band Width

    # Add Percentage Bollinger Bands
        # <0 if below low band
        # [0, 0.5] if between low and mid band
        # [0.5, 1] if between mid and high band
        # >1 if above high band
    df['bb_bbp'] = bb.bollinger_pband() 
    

In [13]:
# ATR
def get_atr(p=14):
    """
    adds atr
    p is period, default 14
    """
    atr = AverageTrueRange(high=df["high"], low=df["low"], close=df["close"], window=p)
    df["atr"]=atr.average_true_range()

## Adding all indicators
Change file name in first and second last line

In [14]:
df = pd.read_csv('data/BTCUSDT-hour-test.csv', sep=',')
df = df.drop('ignore', axis=1)
df = dropna(df)

get_sma()
get_macd()
get_rsi()
get_bb()
get_atr()

df.dropna(inplace=True)
filename = "data/BTCUSDT-" + "hour-test" + "-features.csv"
df.to_csv(filename, index=False)

In [158]:
df

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,...,macd,rsi,bb_bbm,bb_bbh,bb_bbl,bb_bbhi,bb_bbli,bb_bbw,bb_bbp,atr
200,2018-03-05,11515.00,11710.00,11415.01,11454.00,15144.231063,1520294399999,1.748670e+08,164318,7668.823714,...,191.911804,59.007457,10533.1750,11810.936738,9255.413262,0.0,0.0,24.261664,0.860327,865.953730
201,2018-03-06,11455.00,11455.00,10555.48,10716.48,29515.572363,1520380799999,3.238422e+08,250306,13482.297196,...,128.321879,51.938843,10596.4995,11774.929509,9418.069491,0.0,0.0,22.241874,0.550907,868.351321
202,2018-03-07,10716.48,10899.00,9389.31,9910.00,50647.671080,1520467199999,5.148080e+08,396721,24022.451231,...,28.740152,45.517703,10591.9950,11780.158111,9403.831889,0.0,0.0,22.435115,0.213004,914.161227
203,2018-03-08,9910.00,10099.00,9060.00,9271.64,41109.473226,1520553599999,3.946413e+08,336514,20257.455838,...,-77.516779,41.178164,10547.5780,11857.226873,9237.929127,0.0,0.0,24.833168,0.012870,923.078282
204,2018-03-09,9267.07,9410.00,8329.00,9227.00,64112.291407,1520639999999,5.670829e+08,462712,31166.414848,...,-144.320936,40.884625,10456.9505,11865.035994,9048.865006,0.0,0.0,26.931092,0.063254,934.358405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,2021-05-04,57169.39,57200.00,53046.69,53200.01,85324.625903,1620172799999,4.705402e+09,2492629,41029.688246,...,173.452903,43.930142,55414.2380,62455.717386,48372.758614,0.0,0.0,25.413972,0.342773,3299.989963
1357,2021-05-05,53205.05,58069.82,52900.00,57436.11,77263.923439,1620259199999,4.327994e+09,2378119,38729.475016,...,297.700446,53.286248,55128.0445,61298.384403,48957.704597,0.0,0.0,22.385484,0.687029,3433.549251
1358,2021-05-06,57436.11,58360.00,55200.00,56393.68,70181.671908,1620345599999,3.992290e+09,2459881,34248.899643,...,298.172399,51.029653,54880.9885,60398.646674,49363.330326,0.0,0.0,20.107722,0.637077,3414.010019
1359,2021-05-07,56393.68,58650.00,55241.63,57314.75,74542.747829,1620431999999,4.239374e+09,2361692,36877.602566,...,345.665857,52.926555,54746.3930,59874.962516,49617.823484,0.0,0.0,18.735735,0.750397,3413.607160
